In [18]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets


In [3]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director
genre_df = pd.read_csv('/opt/ml/input/data/train/genres.tsv', sep='\t') # item-genre(name)
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

# 아이템별 가장 인기있는 감독 이름(대표 감독)

In [4]:
director_info = pd.read_csv('/opt/ml/input/fighting/FE/director/director_max_sum_intercnt.csv')

In [5]:
director_info

,director,dir_max_cnt,dir_sum_cnt
0,nm0000005,1383,6360
1,nm0000019,905,3290
2,nm0000033,5953,36859
3,nm0000036,761,1835
4,nm0000037,2493,2814
...,...,...,...
1336,nm2648685,247,294
1337,nm2676052,906,1563
1338,nm2879822,251,471
1339,nm9054338,1015,1380


In [7]:
# director 결측 item 감독을 'nm0000000'로 채워주기
no_dir_item_list = list(set(main_df['item']) - set(director_df['item']))
no_dir_item_df = pd.DataFrame([x for x in zip(no_dir_item_list, ['nm0000000']*len(no_dir_item_list))])
no_dir_item_df.columns=director_df.columns # 컬럼명 동일하게
director_df = pd.concat([director_df, no_dir_item_df]) # 기존 director_df 뒤에 감독없는 영화 추가

In [8]:
director_df

,item,director
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005
...,...,...
1299,73681,nm0000000
1300,32728,nm0000000
1301,32743,nm0000000
1302,106491,nm0000000


In [11]:
df = director_df.merge(director_info, how='left', on='director')

In [16]:
item_list = director_df['item'].unique()

In [17]:
item_list

array([  1237,   5147,   7327, ...,  32743, 106491,   8191])

In [29]:
result = []
for item in tqdm(item_list):
    tmp = df[df['item']==item]
    # 'dir_max_cnt' 열에서 가장 큰 값과 그 값을 가진 행을 찾기
    dir_maxsum_cnt = tmp['dir_sum_cnt'].max()
    dir_maxsum_row = df[df['dir_sum_cnt'] == dir_maxsum_cnt]

    # 'director' 열에서 'dir_max_cnt' 열에 가장 큰 값을 가진 행에 해당하는 값 찾기
    director = dir_maxsum_row['director'].values[0]
    result.append((item, director))

100%|██████████| 6807/6807 [00:04<00:00, 1370.72it/s]


In [33]:
result.columns = ['item', 'most_popular_dir']

In [34]:
result = pd.DataFrame(result)
result.to_csv('item_most_popular_director.csv', index=False)

In [35]:
result

,item,most_popular_dir
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005
...,...,...
6802,73681,nm0000000
6803,32728,nm0000000
6804,32743,nm0000000
6805,106491,nm0000000
